In [1]:
import os
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import keras.utils as image
from keras.models import model_from_json

%matplotlib inline

Using TensorFlow backend.


Se não funcionar (`AttributeError`), verificar se a versão do `h5py` é 2.x, pois isso pode causar incompatibilidade.

In [ ]:
# Carregando o modelo treinado (model.json), feito no arquivo model.ipynb
with open('model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model.h5")

print("Loaded model from disk.")

Loaded model from disk.


In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

training_set = train_datagen.flow_from_directory('manipulated_data/images',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 6845 images belonging to 6 classes.


In [22]:
from tensorflow.keras.preprocessing import image

test_image = image.load_img('manipulated_data/images/pattern/0de1181f06cf4b2a8a22bfebc80bf473.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

result = loaded_model.predict(test_image)
training_set.class_indices

{'animal_print': 0,
 'checkers': 1,
 'floral': 2,
 'pattern': 3,
 'solid': 4,
 'stripes': 5}

In [23]:
index = np.where(result[0] == 1)[0][0]

In [24]:
predicted_category = list(training_set.class_indices.keys())[list(training_set.class_indices.values()).index(index)]
predicted_category

'pattern'

### Teste

In [ ]:
folder_dir = "manipulated_data/images/animal_print"
dict_classified = {"id": [], "file_name": [], "Details": []}

for id, filename in enumerate(os.listdir(folder_dir)):
    test_image = image.load_img(os.path.join(folder_dir, filename),
                                target_size=(64, 64))

    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = loaded_model.predict(test_image)

    predicted_category = ""

    if len(np.where(result[0] == 1)[0]) >= 1:
        index = np.where(result[0] == 1)[0][0]
        predicted_category = list(training_set.class_indices.keys())[list(
            training_set.class_indices.values()).index(index)]

    dict_classified["id"].append(id)
    dict_classified["file_name"].append(filename)
    dict_classified["Details"].append(predicted_category)

df_classified = pd.DataFrame(dict_classified)

In [54]:
details_classified = df_classified.Details.value_counts().to_dict()
accuracy = details_classified["animal_print"] / (len(os.listdir(folder_dir)))
print(f"Acurácia para a estampa 'animal_print': {accuracy*100:.2f}%")

Acurácia para a estampa 'animal_print': 36.43%
